<a href="https://colab.research.google.com/github/WDSEatBNL/Intro-to-Machine-Learning-and-AI-Code/blob/master/Machine_Learning_SciKit_Type_in_Categories.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import all necessary Python libraries required for the project.

These include `skimage` for image processing, `matplotlib.pyplot` for plotting, numpy for numerical operations, `joblib` for saving/loading Python objects, `os` for interacting with the operating system, `collections.Counter` for counting items, and `sklearn` modules for machine learning (`SGDClassifier`, `StandardScaler`, `train_test_split`).

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from skimage import io
import os
import ipywidgets as widgets
from IPython.display import display, Image, clear_output

Use the `!git clone` command to download a GitHub repository named 'Intro-to-Machine-Learning-and-AI-Files'. This repository is expected to contain the image datasets used for training and testing the machine learning model.

In [ ]:
!git clone https://github.com/WDSEatBNL/Intro-to-Machine-Learning-and-AI-Files

Read the files from the "images" folder (found in "content/Intro-to-Machine-Learning-and-AI-Files) and ask the user to name the images based on the categories from their card game

***Instructions:*** After running the cell below, you will see an image, an input box, and a submit button. Enter your label in the input box and click 'Submit Label' to proceed to the next image.

In [ ]:
IMG_HEIGHT = 288
IMG_WIDTH = 288
BATCH_SIZE = 32

train_data_dir = r'/content/Intro-to-Machine-Learning-and-AI-Files/images'
image_files = [f for f in os.listdir(train_data_dir) if f.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp'))]

collected_image_labels = {}
current_image_index = 0

image_widget = widgets.Image(width=300)
label_input = widgets.Text(description='Label:')
submit_button = widgets.Button(description='Submit Label')
output_area = widgets.Output()

def display_next_image():
    global current_image_index
    if current_image_index < len(image_files):
        current_image_file = image_files[current_image_index]
        filepath = os.path.join(train_data_dir, current_image_file)

        with open(filepath, 'rb') as f:
            image_data = f.read()
        image_widget.value = image_data
        label_input.value = '' # Clear previous input
        label_input.placeholder = f"Enter label for {current_image_file}"
        with output_area:
            clear_output(wait=True)
            print(f"Labeling image {current_image_index + 1}/{len(image_files)}: {current_image_file}")
    else:
        with output_area:
            clear_output(wait=True)
            print("Labeling complete!")
            print("Collected labels:")
            for filename, label in collected_image_labels.items():
                print(f"  {filename}: {label}")
        submit_button.disabled = True
        label_input.disabled = True

def on_submit_button_clicked(b):
    global current_image_index
    current_image_file = image_files[current_image_index]
    label = label_input.value.strip()
    if label:
        collected_image_labels[current_image_file] = label
        current_image_index += 1
        display_next_image()
    else:
        with output_area:
            print("Please enter a label before submitting.")

submit_button.on_click(on_submit_button_clicked)

display(widgets.VBox([
    image_widget,
    widgets.HBox([label_input, submit_button]),
    output_area
]))
display_next_image()


Take the text labels you provided for your images, identifies all the unique categories, and then creates a numerical mapping.

Then sort your image filenames and use this mapping to convert each image's text label into a corresponding integer. This process is essential because machine learning models require numerical inputs for training.

In [ ]:
all_string_labels = list(collected_image_labels.values())
class_names = sorted(list(set(all_string_labels)))

label_to_index = {label: i for i, label in enumerate(class_names)}

sorted_image_filenames = sorted(image_files)
ordered_integer_labels = [label_to_index[collected_image_labels[filename]] for filename in sorted_image_filenames]

print(f"Unique class names (sorted): {class_names}")
print(f"Number of classes: {len(class_names)}")

This cell is responsible for preparing your image data for the scikit-learn model. It loads and resizes each image, flattens it into a one-dimensional array, then combines all flattened images into a single dataset.

It then splits the data into a training set and a validation set (i.e., set aside 20% of the images for testing).

In [ ]:
image_data = []
for filename in sorted_image_filenames:
    filepath = os.path.join(train_data_dir, filename)
    # Load image and resize
    image = io.imread(filepath)
    image = tf.image.resize(image, (IMG_HEIGHT, IMG_WIDTH)).numpy() # Use tf.image.resize for consistent resizing

    # Flatten the image for scikit-learn
    image_data.append(image.flatten())

X = np.array(image_data)
y = np.array(ordered_integer_labels)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y # stratify helps maintain class proportions
)

# Scale features (important for MLPClassifier)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("Data preparation for scikit-learn complete:")
print(f"X_train_scaled shape: {X_train_scaled.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"X_test_scaled shape: {X_test_scaled.shape}")
print(f"y_test shape: {y_test.shape}")

Set up the model for training: specify a hidden layer with 100 neurons, set the maximum number of iterations (max_iter), and set a random_state for reproducibility.

In [ ]:
model = MLPClassifier(hidden_layer_sizes=(100,), max_iter=1000, random_state=42, verbose=True)

print("Scikit-learn MLPClassifier model created.")
print("Model parameters:")
print(model.get_params())

Train the model and assign a category name to each image in the validation set. This is done many times, showing each iteration and the loss value. These iterations are called epochs. The model will rerun until the loss improvement is less than 0.0001 between epochs.

*   **Higher Loss:** Means the model's predictions are far from the actual values, indicating poor performance.
*   **Lower Loss:** Means the model's predictions are closer to the actual values, indicating better performance.

In [ ]:
print("Training the MLPClassifier model...")
model.fit(X_train_scaled, y_train)

print("Making predictions on the test set...")
predicted_classes = model.predict(X_test_scaled)

# Map integer predictions back to class names for readability
predicted_class_names = [class_names[i] for i in predicted_classes]
actual_class_names = [class_names[i] for i in y_test]

print("Training and prediction complete.")

Show each validation image with its predicted category

In [ ]:
fig = plt.figure(figsize=(12, 12))
num_test_images = len(predicted_class_names)

for i in range(num_test_images):
    ax = plt.subplot(4, int(np.ceil(num_test_images/4)), i + 1)

    # Get the original (unscaled, unflattened) image from X_test
    # X_test contains the flattened images, so we need to reshape them
    image_to_display = X_test[i].reshape(IMG_HEIGHT, IMG_WIDTH, 3).astype('uint8')

    plt.imshow(image_to_display)
    plt.title(f"{predicted_class_names[i]}")
    plt.axis("off")

plt.tight_layout()
plt.show()

Print out the accuracy of the model in the form of percent correctly identified

In [ ]:
print('Percentage correct: ', 100*np.sum(y_test == predicted_classes)/len(predicted_classes))